**Zaliczenie części Wprowadzenie do Sparka**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=691945c2a32769fed067d500cbac6e879771d921fbd0b034a4cf48dcd64f2c79
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


Zbior danych to oceny filmow. Dane pochodza z projektu https://movielens.org/
Wiecej informacji o danych mozna znalezc pod adresem: http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html

In [3]:
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataFrame_add').master('local[*]').getOrCreate()

data_path = '/'

Do rozważenia: Zalecanym formatem przechowywania danych jest Apache Parquet.
Wiecej o tym formacie danych mozna sie dowiedziec np. tutaj: https://parquet.apache.org/.
W skrocie, tak strona projektu podsumowuje misje stojaca za tym formatem:
"We created Parquet to make the advantages of compressed, efficient columnar data representation available to any project in the Hadoop ecosystem".

Przyjrzyj sie danym i przeanalizuj.

Sugestie:
1. Z jakich kolumn skladaja sie wczytane powyzej zbiory?
2. Informacje o ilu filmach znajduja sie w dostarczonym zbiorze?
3. Oceny ilu uzytkownikow znajduja sie w zbiorze?
4. Czy w zbiorze znajduja sie braki danych?
5. Ile filmow nie ma ocen? Ktore filmy nie maja ocen?
6. Ktory film ma najlepsza srednia ocen? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.
7. Jaki procent filmow ma tylko maksymalne oceny?
8. Ktory film na najwyzsza minimalna ocene? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.
9. Jaki jest rozklad ocen?
10. Ile jest filmow zaklasyfikowanych jako dokumentalne 'documentary'?
11. Ktory z filmow dokumentalnych z conajmniej 10 glosami ma najwysza srednia ocene?
12. Jakie sa roznice pomiedzy liczba filmow w zbiorze z roku na rok? Zaloz, ze timestamp reprezentuje liczbe sekund od roku 1960.
13. Ile srednio kategorii przypisanych jest do 1 filmu? Ktory film ma najwiecej kategorii (+co to za kategorie)?

In [46]:
#Wczytanie zbiorów

links_df = spark.read.csv('links.csv', header=True, inferSchema=True)
movies_df = spark.read.csv('movies.csv', header=True, inferSchema=True)
ratings_df = spark.read.csv('ratings.csv', header=True, inferSchema=True)
tags_df = spark.read.csv('tags.csv', header=True, inferSchema=True)

In [9]:
#1. Z jakich kolumn składają się zbiory?
print(f"Columns in links.csv: {pd.read_csv('links.csv').columns.to_list()}")
print(f"Columns in movies.csv: {pd.read_csv('movies.csv').columns.to_list()}")
print(f"Columns in ratings.csv: {pd.read_csv('ratings.csv').columns.to_list()}")
print(f"Columns in tags.csv: {pd.read_csv('tags.csv').columns.to_list()}")

Columns in links.csv: ['movieId', 'imdbId', 'tmdbId']
Columns in movies.csv: ['movieId', 'title', 'genres']
Columns in ratings.csv: ['userId', 'movieId', 'rating', 'timestamp']
Columns in tags.csv: ['userId', 'movieId', 'tag', 'timestamp']


In [11]:
#2. Informacje o ilu filmach znajduja sie w dostarczonym zbiorze?
print(f"links.csv: {pd.read_csv('links.csv').shape}")
print(f"movies.csv: {pd.read_csv('movies.csv').shape}")
print(f"ratings.csv: {pd.read_csv('ratings.csv').shape}")
print(f"tags.csv: {pd.read_csv('tags.csv').shape}")

links.csv: (9742, 3)
movies.csv: (9742, 3)
ratings.csv: (100836, 4)
tags.csv: (3683, 4)


In [23]:
#3. Oceny ilu uzytkownikow znajduja sie w zbiorze?
pd.read_csv('ratings.csv')['userId'].drop_duplicates().count()

610

In [39]:
#4. Czy w zbiorze znajduja sie braki danych?
import os

csvs = [file for file in os.listdir('/content') if os.path.splitext(file)[1] == '.csv']

missing_values = 0
for csv in csvs:
  na = pd.read_csv(csv).isnull().sum().sum()
  if na != 0:
    print(f'{na} braków danych w {csv}')
  missing_values += na

print(f'Łącznie {missing_values} braków danych w całym zbiorze')

8 braków danych w links.csv
Łącznie 8 braków danych w całym zbiorze


In [47]:
#5. Ile filmow nie ma ocen? Ktore filmy nie maja ocen?

movies_with_ratings = movies_df.join(ratings_df, on=['movieId'], how='left')
movies_with_no_ratings = movies_with_ratings.filter(movies_with_ratings.rating.isNull())

# Filmy bez ocen
print("Ilość filmów bez ocen:", movies_with_no_ratings.count())
movies_with_no_ratings.select("movieId", "title").show()

Ilość filmów bez ocen: 18
+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|   1076|Innocents, The (1...|
|   2939|      Niagara (1953)|
|   3338|For All Mankind (...|
|   3456|Color of Paradise...|
|   4194|I Know Where I'm ...|
|   5721|  Chosen, The (1981)|
|   6668|Road Home, The (W...|
|   6849|      Scrooge (1970)|
|   7020|        Proof (1991)|
|   7792|Parallax View, Th...|
|   8765|This Gun for Hire...|
|  25855|Roaring Twenties,...|
|  26085|Mutiny on the Bou...|
|  30892|In the Realms of ...|
|  32160|Twentieth Century...|
|  32371|Call Northside 77...|
|  34482|Browning Version,...|
|  85565|  Chalet Girl (2011)|
+-------+--------------------+



In [48]:
#6. Ktory film ma najlepsza srednia ocen? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.

from pyspark.sql.functions import col, avg, count

# Średnie oceny dla filmów
movie_ratings = ratings_df.groupBy("movieId").agg(avg("rating").alias("average_rating"), count("rating").alias("num_ratings"))

# Film z najlepszą średnią oceną
best_rated_movie = movie_ratings.orderBy(col("average_rating").desc(), col("num_ratings").desc()).limit(1)

# Znalezienie tytułu najlepiej ocenionego filmu
best_rated_movie.show()
best_rated_movie_title = best_rated_movie.join(movies_df, on=["movieId"], how="left")
best_rated_movie_title.select("title", "average_rating").show()

+-------+--------------+-----------+
|movieId|average_rating|num_ratings|
+-------+--------------+-----------+
|     53|           5.0|          2|
+-------+--------------+-----------+

+---------------+--------------+
|          title|average_rating|
+---------------+--------------+
|Lamerica (1994)|           5.0|
+---------------+--------------+



In [82]:
# Znalezienie liczby ocen dla każdego filmu
movie_ratings_count = ratings_df.groupBy("movieId").count()

# Znalezienie filmów, które mają tylko maksymalne oceny
max_rated_movies = movie_ratings_count.filter(col("rating") == 5.0)
num_max_rated_movies = max_rated_movies.count()

# Znalezienie liczby wszystkich filmów
total_movies = ratings_df.select("movieId").distinct().count()

# Obliczenie procentu filmów z tylko maksymalnymi ocenami
percentage = (num_max_rated_movies / total_movies) * 100

# Wyświetlenie wyniku
print(f"Procent filmów z tylko maksymalnymi ocenami: {percentage:.2f}%")

AnalysisException: ignored

In [65]:
from pyspark.sql.functions import max
ratings_df.select(max(ratings_df['rating'])).show()

+-----------+
|max(rating)|
+-----------+
|        5.0|
+-----------+



In [68]:
type(max_rating)

NoneType

In [89]:
# Znalezienie filmów, które mają tylko ocenę 5
movie_ratings = ratings_df.groupBy("movieId").agg(count("rating").alias("num_ratings"), max("rating").alias("max_rating"))

# Wczytanie pliku CSV z informacjami o filmach
movies_info_df = spark.read.csv("movies.csv", header=True, inferSchema=True)

# Połączenie informacji o filmach z wynikami ocen
result_df = movies_info_df.join(movies_with_only_rating_5, on="movieId", how="inner")

# Wyświetlenie wyników
result_df.count()

print(f'{result_df.count()/movies_df.count()}')

0.02966536645452679


In [85]:
max_rated_movies.show()

+-------+----------+
|movieId|max_rating|
+-------+----------+
|   1580|       5.0|
|   2366|       5.0|
|   3175|       5.0|
|   1088|       5.0|
|  32460|       5.0|
|  44022|       4.5|
|  96488|       4.5|
|   1238|       5.0|
|   1342|       4.0|
|   1591|       5.0|
|   1645|       5.0|
|   4519|       4.0|
|   2142|       3.5|
|    471|       5.0|
|   3997|       3.0|
|    833|       4.0|
|   3918|       5.0|
|   7982|       4.5|
|   1959|       5.0|
|  68135|       4.5|
+-------+----------+
only showing top 20 rows



In [76]:
movies_df.distinct().count()

9742

In [98]:
#8. Ktory film na najwyzsza minimalna ocene? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.

# Znalezienie minimalnej oceny dla każdego filmu
movie_min_ratings = ratings_df.groupBy('movieId').agg(count('rating'), min('rating'))

AssertionError: ignored